# 📦 Model Serving & Deployment (MLOps)


En este notebook quiero aprender cómo hacer que un modelo de machine learning, entrenado previamente, esté disponible para que otras personas o aplicaciones lo puedan usar.

Para ello vamos a:

✅ Servir un modelo como una **API REST** usando **FastAPI**  
✅ Cargar un modelo entrenado y utilizarlo para hacer predicciones desde la API  
✅ Probar el endpoint de forma sencilla desde Python  
✅ Entender conceptos clave sobre el **deployment de modelos en producción**






## 🧠 ¿Qué significa "servir un modelo"?

Cuando entreno un modelo en un entorno como Jupyter o Google Colab, solo yo puedo usarlo desde ahí. Si alguien más quiere utilizarlo (por ejemplo, desde una app móvil o una página web), necesita que ese modelo esté disponible **desde internet**.

Para eso lo que hacemos es **servir el modelo como una API web**.  
Esto quiere decir que:

- Exponemos una función (por ejemplo, `predecir_diabetes`) mediante una URL
- Cualquier sistema puede enviar datos a esa URL
- El modelo responderá con la predicción

Esto es la base de muchos productos que usan machine learning.




## 🔧 Herramientas que vamos a usar

| Herramienta     | ¿Para qué sirve?                          |
|------------------|--------------------------------------------|
| **FastAPI**      | Para crear APIs web modernas y fáciles en Python |
| **Uvicorn**      | Servidor web que ejecuta FastAPI           |
| **Pickle**       | Para guardar y cargar modelos entrenados   |
| **requests**     | Para hacer llamadas HTTP desde Python y probar la API





## 📈 Caso práctico: Predicción de diabetes

Vamos a trabajar con un modelo que predice si una persona tiene diabetes, usando el dataset clásico "Pima Indians Diabetes".

El modelo será una regresión logística entrenada con `scikit-learn`, y lo serviremos como una API RESTful usando `FastAPI`.

### 1. Entrenar y guardar el modelo

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pickle

# 1. Cargamos el dataset
df = pd.read_csv("https://raw.githubusercontent.com/selva86/datasets/master/PimaIndiansDiabetes.csv")

# 2. Separar en features (X) y target (y)
X = df.drop("diabetes", axis=1)
y = df["diabetes"]

# 3. Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Entrenar el modelo
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 5. Guardar modelo entrenado como archivo .pkl
with open("diabetes_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Modelo guardado correctamente como 'diabetes_model.pkl'")

### 2. Crear una API para servir el modelo

Ahora vamos a usar **FastAPI** para crear un servidor que cargue el modelo y escuche peticiones. Cuando alguien mande datos, el modelo responderá con una predicción.

La API tendrá:

- Un endpoint básico (`/`) para verificar que todo funciona  
- Un endpoint de predicción (`/predict/`) que recibe los datos y devuelve el resultado del modelo


In [ ]:
# main.py

from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import pickle

# 1. Cargar modelo entrenado desde el archivo .pkl
with open("diabetes_model.pkl", "rb") as f:
    model = pickle.load(f)

# 2. Crear app con FastAPI
app = FastAPI(title="API para predecir diabetes")

# 3. Crear clase para validar la entrada del usuario
class PatientData(BaseModel):
    pregnancies: int
    glucose: float
    bloodpressure: float
    skinthickness: float
    insulin: float
    bmi: float
    diabetespedigreefunction: float
    age: int

# 4. Endpoint base
@app.get("/")
def root():
    return {"message": "API funcionando correctamente"}

# 5. Endpoint de predicción
@app.post("/predict/")
def predict(data: PatientData):
    input_data = np.array([[
        data.pregnancies, data.glucose, data.bloodpressure,
        data.skinthickness, data.insulin, data.bmi,
        data.diabetespedigreefunction, data.age
    ]])

    prediction = model.predict(input_data)[0]
    resultado = "Diabético" if prediction == 1 else "No diabético"

    return {"resultado": resultado}

### 3: Probar la API

Para verificar que la API funciona correctamente, podemos escribir un script en Python que mande datos como si fuéramos una app externa.

Esto es útil para comprobar que el modelo está sirviendo bien las predicciones.

In [ ]:
# test_request.py

import requests

# Dirección local de la API
url = "http://127.0.0.1:8000/predict/"

# Datos de prueba (como si vinieran de una app o frontend)
patient = {
    "pregnancies": 3,
    "glucose": 140,
    "bloodpressure": 75,
    "skinthickness": 23,
    "insulin": 100,
    "bmi": 30.5,
    "diabetespedigreefunction": 0.45,
    "age": 28
}

# Enviar petición POST
response = requests.post(url, json=patient)
print(response.json())

### 4: Ejecutar la API localmente

Para lanzar el servidor de FastAPI, usamos el siguiente comando:

```bash
uvicorn main:app --reload
```

Esto inicia la API en modo "desarrollo" y podrás ver los endpoints en:

http://127.0.0.1:8000


### 5: Dockerizar la API (opcional)

Para poder desplegar nuestra app en cualquier sitio (nube, servidor, etc.), es buena práctica crear una imagen Docker. Así nos aseguramos de que todo funcione igual sin importar el entorno.

**Ejemplo de Dockerfile:**

```dockerfile
FROM python:3.9

WORKDIR /app

COPY requirements.txt .
RUN pip install -r requirements.txt

COPY . .

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]



**¿Y cómo se despliega esto?**

Una vez que tienes la API funcionando y (opcionalmente) dockerizada, puedes subirla a:

- **Render.com** → Muy sencillo para apps FastAPI
- **Hugging Face Spaces** → Ideal si usas `Gradio` o `Streamlit`
- **AWS/GCP/Azure** → Soluciones empresariales escalables
- **Railway.app** o **Fly.io** → Buenas opciones para prototipos rápidos


## Conclusiones

- Ya sé cómo entrenar un modelo, guardarlo y luego servirlo como API con FastAPI
- Este es un paso esencial para convertir un experimento de ML en un **producto real**
- Aprendí a validar inputs, cargar modelos guardados, y trabajar con predicciones desde una app web

### ¿Siguientes pasos?

- Añadir control de errores
- Conectarlo a una base de datos
- Usar `Gradio` o `Streamlit` para crear una interfaz visual
- Desplegarlo en la nube



## (ANEXO) Próximos pasos

### Control de errores

Cuando hacemos una API, no podemos confiar en que los usuarios manden datos perfectos. Imagina que alguien manda edad="hola"... se rompe todo. Así que hay que prevenir errores.

Con FastAPI y pydantic, ya tenemos validación básica, pero podemos mejorarlo.



**Ejemplo: Controlar si falta un campo o si el valor no es numérico**

In [ ]:
from fastapi import HTTPException

@app.post("/predict/")
def predict(data: PatientData):
    try:
        input_data = np.array([[ ... ]])  # igual que antes
        prediction = model.predict(input_data)[0]
        resultado = "Diabético" if prediction == 1 else "No diabético"
        return {"resultado": resultado}
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error en la predicción: {str(e)}")

### CI/CD: Integración y Despliegue automático

CI/CD significa:

- CI (Integración Continua): testear tu código automáticamente cuando haces cambios.
- CD (Despliegue Continuo): subir automáticamente tu app a producción cuando actualizas el repositorio (por ejemplo, en GitHub).



**Ejemplo: CI/CD con GitHub Actions**

1. Crea un archivo en .github/workflows/deploy.yml

```
name: Deploy API

on:
  push:
    branches:
      - main

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - name: Set up Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.9'
      - name: Install dependencies
        run: pip install -r requirements.txt
      - name: Run tests
        run: python -m unittest discover tests
```



2. Esto testea tu código cada vez que haces git push al repositorio.

  Si usas Render.com o Railway.app, puedes conectar tu repo de GitHub y el despliegue se hace automáticamente cada vez que subes código nuevo.

### Conectar la API a una base de datos



Supongamos que quiero guardar cada predicción que hago: quién la hizo, cuándo, y qué datos mandó. Para eso necesito una base de datos.

**Ejemplo sencillo con SQLite (base de datos ligera en archivo local)**

In [ ]:
import sqlite3

# Crear base de datos (solo la primera vez)
conn = sqlite3.connect("predicciones.db")
cursor = conn.cursor()

# Crear tabla
cursor.execute("""
CREATE TABLE IF NOT EXISTS predicciones (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    edad INTEGER,
    resultado TEXT
)
""")
conn.commit()
conn.close()

Y luego dentro del endpoint:

In [ ]:
# Guardar predicción
conn = sqlite3.connect("predicciones.db")
cursor = conn.cursor()
cursor.execute("INSERT INTO predicciones (edad, resultado) VALUES (?, ?)", (data.age, resultado))
conn.commit()
conn.close()

> Más adelante puedes usar PostgreSQL si lo subes a la nube.

### Interfaz visual con Streamlit o Gradio

Sirve para que cualquier persona (sin saber código) pueda probar tu modelo desde un navegador.

In [ ]:
import gradio as gr
import pickle
import numpy as np

with open("diabetes_model.pkl", "rb") as f:
    model = pickle.load(f)

def predecir(preg, gluc, presion, piel, insulina, bmi, pedigree, edad):
    datos = np.array([[preg, gluc, presion, piel, insulina, bmi, pedigree, edad]])
    pred = model.predict(datos)[0]
    return "Diabético" if pred == 1 else "No diabético"

demo = gr.Interface(
    fn=predecir,
    inputs=[
        gr.Number(label="Embarazos"),
        gr.Number(label="Glucosa"),
        gr.Number(label="Presión arterial"),
        gr.Number(label="Grosor piel"),
        gr.Number(label="Insulina"),
        gr.Number(label="BMI"),
        gr.Number(label="Índice genético"),
        gr.Number(label="Edad"),
    ],
    outputs="text",
    title="Predicción de Diabetes"
)

demo.launch()

> Puedes subir esta interfaz directamente a HuggingFace Spaces sin necesidad de servidores.

### Resumen Visual

```
[ Modelo Entrenado ]
        ↓
[ API FastAPI (localhost) ]
        ↓
[ Docker / Render / HuggingFace Spaces ]
        ↓
[ Producción con CI/CD ]
        ↓
[ Interfaz visual / Apps externas ]

```
